In [54]:
import os
import numpy as np


In [55]:
from classes.money_line import MoneyLine
from classes.parlay import Parlay
from classes.parlay_system import ParlaySystem

In [56]:
match_a_team_1 = MoneyLine(event="KC", 
                           bet_amount=100, 
                        #    money_line_odds=100) # with profit boost
                           money_line_odds=-550)

match_a_team_1_with_boost = MoneyLine(event="KC", 
                                      bet_amount=100, 
                                      money_line_odds=100) # with profit boost


extra_parlay = Parlay(money_line_arr=[match_a_team_1_with_boost], bet_amount=100)

# match_a_team_1.df_stats


In [57]:
match_a_team_2 = MoneyLine(event="ny",
                           bet_amount=100,
                           money_line_odds=400)
# match_a_team_2.df_stats

In [58]:
match_b_team_1 = MoneyLine(event="pats",
                           bet_amount=100,
                           money_line_odds=-190)

# match_b_team_1.df_stats


In [59]:
match_b_team_2 = MoneyLine(event="panthers",
                           bet_amount=100,
                           money_line_odds=160)

# match_b_team_2.df_stats

In [60]:
match_c_team_1 = MoneyLine(event="ATL",
                           bet_amount=100,
                           money_line_odds=215)

# match_c_team_1.df_stats

In [61]:
match_c_team_2 = MoneyLine(event="saints",
                           bet_amount=100,
                           money_line_odds=-265)

# match_c_team_2.df_stats

In [62]:
binaries = [
    [match_a_team_1, match_a_team_2],
    [match_b_team_1, match_b_team_2],
    [match_c_team_1, match_c_team_2]
]

In [63]:
parlay_system = ParlaySystem(binaries=binaries,
                             target_profit=0.075,
                             bounds=(0.01, 50),
                             extra_parlays=[match_a_team_1_with_boost])

results = parlay_system.slsqp_solver(profit_type='avg_implied_profit')
# results = parlay_system.slsqp_solver(profit_type='avg_profit')


            constraints:
            [{'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x13612d290>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x13612d8c0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x13612d4d0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x1360db4d0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x136002ef0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x1360b88c0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x136161f80>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x136161ef0>}, {'type': 'ineq', 'fun': <function ParlaySystem.slsqp_solver.<locals>.<lambda> at 0x136161e60>}]

            bounds:
            ((0.01, 50), (0.01, 50), (0.01, 50), (0.01, 50), (0.01, 50), (0.

In [64]:
results.head(10)
results.to_csv('/Users/ramp/Desktop/projects/sports_book2/data/nfl_profit_boost.csv', header=True, index=False)

# results_implied_profit.head(10)

In [65]:
print(f'''
    Total Bet Amount: {results['bet'].sum()}
    Total Implied Probability: {results['implied_prob'].sum()}
    Average Profit: {results['profit'].mean()}
''')


    Total Bet Amount: 0.1285
    Total Implied Probability: 1.6350829840274264
    Average Profit: 0.037000000000000005



In [66]:
results['expected_value'] = results['implied_prob'] * results['profit']
results[['event', 'bet', 'dec_odds', 'implied_prob', 'payout','profit', 'expected_value']].head(10)

,event,bet,dec_odds,implied_prob,payout,profit,expected_value
0,ATL_ny_panthers,0.0100,40.9500,0.024420,0.4095,0.281,0.006862
1,ATL_ny_pats,0.0100,24.0395,0.041598,0.2404,0.112,0.004659
2,ny_panthers_saints,0.0114,17.9057,0.055848,0.2034,0.075,0.004189
3,ny_pats_saints,0.0194,10.5114,0.095135,0.2034,0.075,0.007135
4,ATL_KC_panthers,0.0210,9.6791,0.103315,0.2034,0.075,0.007749
5,ATL_KC_pats,0.0196,5.6821,0.175993,0.1115,-0.017,-0.002992
6,KC_panthers_saints,0.0171,4.2322,0.236281,0.0722,-0.056,-0.013232
7,KC_pats_saints,0.0100,2.4845,0.402493,0.0248,-0.104,-0.041859
8,KC,0.0100,2.0000,0.500000,0.0200,-0.108,-0.054000


In [67]:

print(f'''
    Total Bet           : {results['bet'].sum()}
    Total Expected value: {results['expected_value'].sum()}
''')


    Total Bet           : 0.1285
    Total Expected value: -0.08148944198411877



In [68]:
all_results = results.loc[results['profit'] != 0.000000000] 
all_results

,event,ml_odds,dec_odds,bet,implied_prob,payout,override,profit,expected_value
0,ATL_ny_panthers,3995.000000,40.9500,0.0100,0.024420,0.4095,0,0.281,0.006862
1,ATL_ny_pats,2303.947368,24.0395,0.0100,0.041598,0.2404,0,0.112,0.004659
2,ny_panthers_saints,1690.566038,17.9057,0.0114,0.055848,0.2034,0,0.075,0.004189
3,ny_pats_saints,951.142006,10.5114,0.0194,0.095135,0.2034,0,0.075,0.007135
4,ATL_KC_panthers,867.909091,9.6791,0.0210,0.103315,0.2034,0,0.075,0.007749
5,ATL_KC_pats,468.205742,5.6821,0.0196,0.175993,0.1115,0,-0.017,-0.002992
6,KC_panthers_saints,323.224700,4.2322,0.0171,0.236281,0.0722,0,-0.056,-0.013232
7,KC_pats_saints,148.451747,2.4845,0.0100,0.402493,0.0248,0,-0.104,-0.041859
8,KC,100.000000,2.0000,0.0100,0.500000,0.0200,0,-0.108,-0.054000


In [69]:
pos_results = results.loc[results['profit'] > -0.005] 
pos_results

,event,ml_odds,dec_odds,bet,implied_prob,payout,override,profit,expected_value
0,ATL_ny_panthers,3995.000000,40.9500,0.0100,0.024420,0.4095,0,0.281,0.006862
1,ATL_ny_pats,2303.947368,24.0395,0.0100,0.041598,0.2404,0,0.112,0.004659
2,ny_panthers_saints,1690.566038,17.9057,0.0114,0.055848,0.2034,0,0.075,0.004189
3,ny_pats_saints,951.142006,10.5114,0.0194,0.095135,0.2034,0,0.075,0.007135
4,ATL_KC_panthers,867.909091,9.6791,0.0210,0.103315,0.2034,0,0.075,0.007749


In [70]:
pos_results['implied_prob'].sum()

0.3203166466419982